### Picking which words are valid for the crossword
The words will be saved in a csv file. alongside any relevant information.

In [19]:
import os
import re
import numpy as np
import pandas as pd
import time
import json

frequency_path = os.path.join(os.getcwd(),'..','data','korpus','korpus_frequency.csv')
wordforms_path = os.path.join(os.getcwd(),'..','data','gabra','wordforms.csv')
lessiku_path = os.path.join(os.getcwd(),'..','data','gabra','lexemes.csv')
final_lexemes_path = os.path.join(os.getcwd(),'..','data','gabra','final','lexemes.csv')


In [20]:
#Load frequency counts of words from korpus
df_frequency = pd.read_csv(frequency_path)

#Load lexemes
df_lexemes = pd.read_csv(lessiku_path, encoding='utf-32')[["lemma","root","sources","phonetic","pos","glosses","norm_freq"]]

In [21]:
df_lexemes = df_lexemes[df_lexemes['sources'] != "['UserFeedback']"]
df_lexemes = df_lexemes.dropna()

In [22]:
df_lexemes = df_lexemes[df_lexemes['norm_freq'] > 15]
df_lexemes = df_lexemes[df_lexemes['lemma'] != "laqa'"]
df_lexemes = df_lexemes[df_lexemes['lemma'] != "nqata'"]

In [23]:
df_lexemes

,lemma,root,sources,phonetic,pos,glosses,norm_freq
22,banda,{'radicals': 'b-n-d-j'},['Spagnol2011'],bɐndɐ,VERB,"[{'gloss': 'to ban', 'examples': []}, {'gloss'...",134.479771
53,basta,{'radicals': 'b-s-t-j'},['Spagnol2011'],bɐstɐ,VERB,"[{'gloss': 'to suffice', 'examples': []}]",65.611044
90,ċempel,{'radicals': 'ċ-m-p-l'},"['Spagnol2011', 'Falzon2013']",ʧɛmpɛl,VERB,"[{'gloss': 'to ring', 'examples': []}, {'gloss...",38.562636
136,fonda,"{'radicals': 'f-n-d-j', 'variant': 2}",['Spagnol2011'],fɔndɐ,VERB,"[{'gloss': 'to deepen', 'examples': []}, {'glo...",210.742479
178,fixkel,{'radicals': 'f-x-k-l'},"['Spagnol2011', 'Falzon2013']",fɪʃkɛl,VERB,"[{'gloss': 'to confuse', 'examples': []}, {'gl...",17.744750
...,...,...,...,...,...,...,...
16633,salva,"{'radicals': 's-l-v-j', 'variant': 1}",['DM2015'],sɐlvɐ,NOUN,"[{'gloss': ""Name of a children's game"", 'examp...",19.642391
16639,salvataġġ,"{'radicals': 's-l-v-j', 'variant': 1}",['DM2015'],sɐlvɐtɐʤʧ,NOUN,"[{'gloss': 'salvage', 'examples': [{'example':...",32.753367
16654,sensiel,{'radicals': 's-n-s-l'},['DM2015'],sɛnsɪːl,NOUN,"[{'gloss': 'sb. who strings things together', ...",64.583990
16673,serv,{'radicals': 's-r-v-j'},['DM2015'],sɛrf,NOUN,"[{'gloss': 'servant', 'examples': [{'example':...",16.854103


##### Parsing glosses and seperating examples

In [24]:
glosses = []
examples = []

for txt in df_lexemes['glosses']:

    result = re.findall('{(.*?)}', txt)
    
    gloss   = [i.split(',')[0] for i in result]
    gloss   = ', '.join([g[10:-1] for g in gloss])
    glosses.append(gloss)
    
#     if len(result[0].split(',')) == 2:
#         example = [i.split(',')[1] for i in result]
#         example = ', '.join([i[14:-1] for i in example])
#         print(example)
#         examples.append(example)
#     else:
#         examples.append('')

df_lexemes['glosses'] = glosses

In [25]:
df_lexemes

,lemma,root,sources,phonetic,pos,glosses,norm_freq
22,banda,{'radicals': 'b-n-d-j'},['Spagnol2011'],bɐndɐ,VERB,"to ban, to ostracise, to exile",134.479771
53,basta,{'radicals': 'b-s-t-j'},['Spagnol2011'],bɐstɐ,VERB,to suffice,65.611044
90,ċempel,{'radicals': 'ċ-m-p-l'},"['Spagnol2011', 'Falzon2013']",ʧɛmpɛl,VERB,"to ring, to call sb. on the phone",38.562636
136,fonda,"{'radicals': 'f-n-d-j', 'variant': 2}",['Spagnol2011'],fɔndɐ,VERB,"to deepen, to plunge, to dive to the bottom, t...",210.742479
178,fixkel,{'radicals': 'f-x-k-l'},"['Spagnol2011', 'Falzon2013']",fɪʃkɛl,VERB,"to confuse, to thwart, to disrupt",17.744750
...,...,...,...,...,...,...,...
16633,salva,"{'radicals': 's-l-v-j', 'variant': 1}",['DM2015'],sɐlvɐ,NOUN,Name of a children's game,19.642391
16639,salvataġġ,"{'radicals': 's-l-v-j', 'variant': 1}",['DM2015'],sɐlvɐtɐʤʧ,NOUN,"salvage, rescue",32.753367
16654,sensiel,{'radicals': 's-n-s-l'},['DM2015'],sɛnsɪːl,NOUN,sb. who strings things together,64.583990
16673,serv,{'radicals': 's-r-v-j'},['DM2015'],sɛrf,NOUN,"servant, serf",16.854103


In [26]:
df_lexemes

,lemma,root,sources,phonetic,pos,glosses,norm_freq
22,banda,{'radicals': 'b-n-d-j'},['Spagnol2011'],bɐndɐ,VERB,"to ban, to ostracise, to exile",134.479771
53,basta,{'radicals': 'b-s-t-j'},['Spagnol2011'],bɐstɐ,VERB,to suffice,65.611044
90,ċempel,{'radicals': 'ċ-m-p-l'},"['Spagnol2011', 'Falzon2013']",ʧɛmpɛl,VERB,"to ring, to call sb. on the phone",38.562636
136,fonda,"{'radicals': 'f-n-d-j', 'variant': 2}",['Spagnol2011'],fɔndɐ,VERB,"to deepen, to plunge, to dive to the bottom, t...",210.742479
178,fixkel,{'radicals': 'f-x-k-l'},"['Spagnol2011', 'Falzon2013']",fɪʃkɛl,VERB,"to confuse, to thwart, to disrupt",17.744750
...,...,...,...,...,...,...,...
16633,salva,"{'radicals': 's-l-v-j', 'variant': 1}",['DM2015'],sɐlvɐ,NOUN,Name of a children's game,19.642391
16639,salvataġġ,"{'radicals': 's-l-v-j', 'variant': 1}",['DM2015'],sɐlvɐtɐʤʧ,NOUN,"salvage, rescue",32.753367
16654,sensiel,{'radicals': 's-n-s-l'},['DM2015'],sɛnsɪːl,NOUN,sb. who strings things together,64.583990
16673,serv,{'radicals': 's-r-v-j'},['DM2015'],sɛrf,NOUN,"servant, serf",16.854103


### Parsing roots

In [27]:
regex = re.compile('[a-zġħċż][a-zġħċż]?-[a-zġħċż][a-zġħċż]?-[a-zġħċż][a-zġħċż]?-?[a-zġħċż]?[a-zġħċż]?',)

regex.findall('b-d-b-t')[0]
df_lexemes['root'] = df_lexemes['root'].apply(lambda s: ''.join(regex.findall(s)))
df_lexemes

,lemma,root,sources,phonetic,pos,glosses,norm_freq
22,banda,b-n-d-j,['Spagnol2011'],bɐndɐ,VERB,"to ban, to ostracise, to exile",134.479771
53,basta,b-s-t-j,['Spagnol2011'],bɐstɐ,VERB,to suffice,65.611044
90,ċempel,ċ-m-p-l,"['Spagnol2011', 'Falzon2013']",ʧɛmpɛl,VERB,"to ring, to call sb. on the phone",38.562636
136,fonda,f-n-d-j,['Spagnol2011'],fɔndɐ,VERB,"to deepen, to plunge, to dive to the bottom, t...",210.742479
178,fixkel,f-x-k-l,"['Spagnol2011', 'Falzon2013']",fɪʃkɛl,VERB,"to confuse, to thwart, to disrupt",17.744750
...,...,...,...,...,...,...,...
16633,salva,s-l-v-j,['DM2015'],sɐlvɐ,NOUN,Name of a children's game,19.642391
16639,salvataġġ,s-l-v-j,['DM2015'],sɐlvɐtɐʤʧ,NOUN,"salvage, rescue",32.753367
16654,sensiel,s-n-s-l,['DM2015'],sɛnsɪːl,NOUN,sb. who strings things together,64.583990
16673,serv,s-r-v-j,['DM2015'],sɛrf,NOUN,"servant, serf",16.854103


### Replacing abbreviations

In [28]:
df_lexemes[df_lexemes['glosses'].str.contains('sth.')]

,lemma,root,sources,phonetic,pos,glosses,norm_freq
223,gawda,g-w-d-j,['Spagnol2011'],gɐʊdɐ,VERB,"to enjoy, to take pleasure in sth., to have an...",88.836085
323,kompla,k-m-pl-j,"['Spagnol2011', 'Falzon2013']",kɔmplɐ,VERB,to continue sth.,774.394060
584,serva,s-r-v-j,"['Spagnol2011', 'Falzon2013']",sɛrvɐ,VERB,"to serve, to be appropriate for sth.",116.426103
693,bidel,b-d-l,"['Spagnol2011', 'Falzon2013']",bɪdɛl,VERB,"to change sth, to alter sth",70.670083
694,biddel,b-d-l,"['Spagnol2011', 'Falzon2013']",bɪddɛl,VERB,"to change sth, to alter sth, to change one's c...",110.352046
...,...,...,...,...,...,...,...
13783,fakkar,f-k-r,['DM2015'],fɐkkɐr,NOUN,that acts to remind of sth.,66.533787
14359,lejliet,l-j-l,['DM2015'],lɛɪlɪːt,NOUN,"on the eve of sth., the night before sth.",23.429650
15111,tħaddim,ħ-d-m,['DM2015'],thɐddɪm,NOUN,"employing of persons, using sth. frequently, ...",41.671879
15985,marbut,r-b-t,['DM2015'],mɐrbʊt,ADJ,"tied up, fastened, committed or duty-bound to ...",189.407055


In [29]:
df_lexemes['glosses'] = df_lexemes['glosses'].apply(lambda s: s.replace('sb.','somebody')
                                                    .replace('sth.','something')
                                                    .replace('sb','somebody')
                                                    .replace('sth','something'))

df_lexemes[df_lexemes['glosses'].str.contains('somebody')]

,lemma,root,sources,phonetic,pos,glosses,norm_freq
90,ċempel,ċ-m-p-l,"['Spagnol2011', 'Falzon2013']",ʧɛmpɛl,VERB,"to ring, to call somebody on the phone",38.562636
554,salva,s-l-v-j,"['Spagnol2011', 'Falzon2013']",sɐlvɐ,VERB,"to save somebody, to rescue, to redeem",63.713402
869,beka,b-k-j,"['Spagnol2011', 'Falzon2013']",bɛkɐ,VERB,"to cry, to shed tears, to mourn somebody",23.196958
906,ċaħad,ċ-ħ-d,"['Spagnol2011', 'Falzon2013']",ʧɐhɐt,VERB,"to deny, to disavow, to disclaim, to adjure, t...",50.843143
977,deher,d-h-r,['Spagnol2011'],dɛːr,VERB,"to appear, to be seen, to resemble something o...",702.492316
...,...,...,...,...,...,...,...
15314,ħallas,ħ-l-s,['DM2015'],hɐllɐs,NOUN,somebody who pays,38.538565
15617,żerriegħ,ż-r-għ,['DM2015'],zɛrrɪːh,NOUN,somebody who scatters seeds or sows,48.752929
15848,tellieq,t-l-q,['DM2015'],tɛllɪːʔ,NOUN,somebody who lets the horses out during horse ...,25.780639
15985,marbut,r-b-t,['DM2015'],mɐrbʊt,ADJ,"tied up, fastened, committed or duty-bound to ...",189.407055


In [30]:
df_lexemes = df_lexemes.reset_index(drop=True)
df_lexemes.to_csv(final_lexemes_path, index=False)